In [1]:
import pandas as pd
import numpy as np
from dateutil import parser
from dateutil import parser
from datetime import timedelta
from datetime import date
from datetime import datetime
from datetime import time
import glob, os
from tqdm import tqdm, tnrange, tqdm_notebook

In [2]:
#station_id:ID(Station-ID),name,:LABEL
df_stations = pd.read_csv('graph_stations.csv')
station_id_list = df_stations['station_id:ID(Station-ID)'].unique()
#event_id:ID(Event-ID),train_id(Train-ID),station_id(Station-ID),timestamp:double,:LABEL
df_events = pd.read_csv('graph_events.csv')

In [3]:
station_list = []
event_list = []
station_index_from_event_id = {}
station_index = 0
for stat_index, stat_row in df_stations.iterrows():
    station_id = stat_row['station_id:ID(Station-ID)']
    station_name = stat_row['name']
    station_list.append((station_index, station_name, station_id))
    station_index += 1

for stat_index, stat_name, stat_id in station_list:
    #print("stat_index" + str(stat_index) + "stat_id" + str(stat_id) + "stat_name" + str(stat_name))
    df_stat_events = df_events.loc[df_events['station_id(Station-ID)'] == stat_id].copy()
    for event_index, event_row in df_stat_events.iterrows():   
        event_id = event_row['event_id:ID(Event-ID)']
        event_ts = event_row['timestamp:double']
        event_date_str = datetime.fromtimestamp(event_ts).strftime('%Y.%m.%d.%H:%M')
        stat_id_event_date_str = str(stat_index)+"."+event_date_str
        event_list.append((station_index, stat_id_event_date_str, event_id))
        station_index_from_event_id[event_id] = station_index
        station_index += 1
station_list.extend(event_list)

In [9]:
columns = ['id', 'name', 'id_orig']
df_stations_events = pd.DataFrame(station_list, columns=columns)
df_stations_events.to_csv('r_stations_events.csv', index=False)

In [5]:
#:START_ID(Event-ID),train_id(Train-ID),travel_time:int,:END_ID(Event-ID),:TYPE
#(Event-ID) = str(train_id)+":"+str(row_to.station_id)+":"+str(int(ts_arr))+":arr"
df_trips = pd.read_csv('graph_trips.csv')

123

In [6]:
connections_list = []
pBarEvent = tqdm_notebook(total=len(df_trips), desc='Events', leave=False)
for trip_index, trip_row in df_trips.iterrows():
    pBarEvent.update(1)
    event_from_id = trip_row[':START_ID(Event-ID)']
    event_to_id = trip_row[':END_ID(Event-ID)']
    travel_time = trip_row['travel_time:int']
    stat_from_id = station_index_from_event_id[event_from_id]
    stat_to_id = station_index_from_event_id[event_to_id]
    connections_list.append((stat_from_id, stat_to_id, int(travel_time)))
pBarEvent.close()

In [7]:
columns = ['st_from', 'st_to', 'weight']
df_connections = pd.DataFrame(connections_list, columns=columns)
df_connections.to_csv('r_connections_travel.csv', index=False)

In [8]:
connections_list = []
#:START_ID(Event-ID),:END_ID(Event-ID),wait_time:int,:TYPE
pBarFrom = tqdm_notebook(total=len(glob.glob("waits\*.csv")), desc='Files', leave=False)
for counter, file in enumerate(glob.glob("waits\*.csv")):
    pBarFrom.update(1)
    df_waits = pd.read_csv(file)
    pBarEvent = tqdm_notebook(total=len(df_waits), desc='Events', leave=False)
    for wait_index, wait_row in df_waits.iterrows():
        pBarEvent.update(1)
        event_from_id = wait_row[':START_ID(Event-ID)']
        event_to_id = wait_row[':END_ID(Event-ID)']
        travel_time = wait_row['wait_time:int']
        stat_from_id = station_index_from_event_id[event_from_id]
        stat_to_id = station_index_from_event_id[event_to_id]
        connections_list.append((stat_from_id, stat_to_id, int(travel_time)))
    pBarEvent.close()
pBarFrom.close()

In [9]:
columns = ['st_from', 'st_to', 'weight']
df_connections = pd.DataFrame(connections_list, columns=columns)
df_connections.to_csv('r_connections_wait.csv', index=False)